# Описание проекта

## Цель проекта

Разработать нейросеть, способную определять температуру на поверхности звёз, исходя из их характеристик. В наличие имеются данные по 240 уже изученным звёздам

## Описание данных

Датасет содержит следующие данные по объектам:
* Относительная светимость `L/Lo` — светимость звезды относительно Солнца.
* Относительный радиус `R/Ro` — радиус звезды относительно радиуса Солнца.
* Абсолютная звёздная величина `Mv` — физическая величина, характеризующая блеск звезды.
* Звёздный цвет (`white`, `red`, `blue`, `yellow`, `yellow-orange` и др.) — цвет звезды, который определяют на основе спектрального анализа.
* Тип звезды:

| Тип звезды | Номер, соответствующий типу |
| ---------- | --------------------------- |
| Коричневый карлик | 0 |
| Красный карлик | 1 |
| Белый карлик | 2 |
| Звёзды главной последовательности | 3 |
| Сверхгигант | 4 |
| Гипергигант | 5 |

* Абсолютная температура `T(K)` — температура на поверхности звезды в Кельвинах.

# Загрузка данных

## Импорт библиотек

In [1]:
import pandas as pd

## Загрузка датасета

In [4]:
try:
    df = pd.read_csv('./datasets/6_class.csv', index_col=0) # local
except:
    df = pd.read_csv('/datasets/6_class_csv', index_col=0)

In [5]:
df

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color
0,3068,0.002400,0.1700,16.12,0,Red
1,3042,0.000500,0.1542,16.60,0,Red
2,2600,0.000300,0.1020,18.70,0,Red
3,2800,0.000200,0.1600,16.65,0,Red
4,1939,0.000138,0.1030,20.06,0,Red
...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,5,Blue
236,30839,834042.000000,1194.0000,-10.63,5,Blue
237,8829,537493.000000,1423.0000,-10.73,5,White
238,9235,404940.000000,1112.0000,-11.23,5,White


# Предобработка данных

# Построение базовой нейронной сети

# Улучшение базовой нейронной сети

# Выводы